In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Proximal Policy Optimization (PPO) 문제

## Q) PPO의 objective function을 최적화하는 코드를 아래(Policy class 내부)에 구현해 주세요. 코드에서 물음표 부분을 채워주시면 됩니다 (총 6개의 물음표).

PPO에서 policy를 optimize하기 위한 objective function은 다음과 같습니다.

$$L(\theta) = E_t[  min(r_t(\theta)A_t, clip(r_t(\theta), 1-\epsilon, 1+\epsilon)A_t) ]$$

위 식에서, 
$$r_t(\theta) = \frac{\pi_\theta(s_t, a_t)}{\pi_{old}(s_t, a_t)},$$

$$A_t = A(s_t, a_t): advantage function$$
에 해당합니다.

PPO는 $L(\theta)$를 최대화하도록 policy $\pi_\theta$의 업데이트를 수행하는 알고리즘입니다.

In [ ]:
import numpy as np

import warnings  
with warnings.catch_warnings():  
    warnings.filterwarnings("ignore",category=FutureWarning)
    import tensorflow.compat.v1 as tf
    tf.disable_eager_execution()

class Policy(object):
    """
    주어진 observation에 대해 수행할 행동의 확률 분포를 Gaussian 분포로 표현하며,
    해당 Guassian 분포의 mean과 variance를 출력한다.
    """

    def __init__(self, obs_dim, act_dim, clipping=0.2):
        """
        Args:
            obs_dim: observation의 dimension
            act_dim: action의 dimensions
            clipping: r_t(\theta)를 clipping 할 범위 (위에 표기된 objective 수식에서의 epsilon에 해당)
        """
        self.obs_ph = tf.keras.layers.Input(obs_dim, name='obs')
        self.act_ph = tf.keras.layers.Input(act_dim, name='act')
        self.advantages_ph = tf.keras.layers.Input((None,), name='advantages')

        h1 = tf.keras.layers.Dense(32, activation="tanh", name="h1")(self.obs_ph)
        h2 = tf.keras.layers.Dense(32, activation="tanh", name="h2")(h1)

        # observation이 주어졌을 때, 수행할 행동에 대한 Gaussian 확률 분포의 mean (observation dependent).
        self.means = tf.keras.layers.Dense(act_dim, name="means", activation="linear")(h2)

        # 수행할 행동의 Gaussian 분포에 대한 전역적 log(variance) 값 (observation independent).
        # 참고로, tf.exp(self.log_vars) 를 수행하면 Gaussian 분포의 variance가 됨.
        self.log_vars = tf.get_variable('log_vars', (act_dim), tf.float32, tf.constant_initializer(-1.0))

        # (observation, action)이 주어졌을 때, 현재 policy가 해당 observation에서 주어진 action을 수행할 확률의 log 값.
        log_p = self._log_prob(self.means, self.log_vars, self.act_ph)

        # 데이터를 수집한 old policy에서 각 observation에 대한,
        # Gaussian 행동 확률 분포의 mean과 log(variance) 입력.
        self.old_means_ph = tf.keras.layers.Input(act_dim, name='old_means')
        self.old_log_vars_ph = tf.keras.layers.Input(act_dim, name='old_log_vars')

        # (observation, action)이 주어졌을 때, 데이터를 수집한 policy(old policy)가
        # 해당 observation에서 주어진 action을 수행할 확률의 log 값.
        log_p_old = self._log_prob(self.old_means_ph, self.old_log_vars_ph, self.act_ph)

        # ===============================================================================================#
        # Your code here (아래의 물음표를 채워주세요.)
        # ===============================================================================================#
        pg_ratio = tf.exp(???? - ????)  # pg_ratio는 PPO objective에서 r_t(\theta)에 해당.
        clipped_pg_ratio = tf.clip_by_value(pg_ratio, 1 - ????, 1 + ????)
        surrogate_objective = tf.minimum(????, ????)  # [hint] A_t의 값으로 self.advantages_ph를 활용.
        # ===============================================================================================#

        self.loss = -tf.reduce_mean(surrogate_objective)

        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        self.train_op = optimizer.minimize(self.loss)

        # 주어진 Gaussian 분포의 mean과 log(variance)로 action을 샘플.
        # action ~ N(mean, variance) 는 action = mean + (starndard_deviation) * e, e ~ N(0, 1) 로 표현 가능.
        self.sampled_act = self.means + tf.exp(self.log_vars / 2.0) * tf.random_normal(shape=(act_dim,))

        self.epochs = 20  # 주어진 데이터에 대해 현재 policy update를 반복할 epoch 수
        self.sess = tf.keras.backend.get_session()

    def _log_prob(self, means, log_vars, x):
        """
        Guassian 분포의 mean과 log(variance)가 주어졌을 때, 특정 값 x의 log 확률을 계산하여 리턴.
        """
        log_prob = -0.5 * tf.reduce_sum(log_vars)
        log_prob += -0.5 * tf.reduce_sum(tf.square(x - means) / tf.exp(log_vars), axis=1)
        return log_prob

    def sample(self, obs):
        """
        주어진 각 observation에 대해 수행할 행동을 policy의 Gaussian 확률분포로부터 샘플링하여 리턴.
        """
        feed_dict = {
            self.obs_ph: obs
        }
        return self.sess.run(self.sampled_act, feed_dict=feed_dict)

    def update(self, observes, actions, advantages, logger):
        """
        환경에서 수집된 데이터(observes, actions, advantages)를 기반으로 현재 policy의 업데이트를 수행.
        """
        feed_dict = {
            self.obs_ph: observes,
        }

        # 수집된 observation에 대해 데이터 수집 policy (old policy)가 나타내는,
        # Gaussian 행동 분포의 mean과 log(variance)를 계산.
        old_means, old_log_vars = self.sess.run([self.means, self.log_vars], feed_dict)

        feed_dict = {
            self.obs_ph: observes,
            self.act_ph: actions,
            self.advantages_ph: [advantages],
            self.old_means_ph: old_means,
            self.old_log_vars_ph: [old_log_vars],
        }

        # clipped surrogate objective를 통해 현재 policy를 update.
        for e in range(self.epochs):
            self.sess.run(self.train_op, feed_dict)
            policy_loss = self.sess.run(self.loss, feed_dict)

        logger.log({'PolicyLoss': policy_loss})

작성한 Policy class가 잘 동작하는지 여부를 아래의 코드로 테스트해 볼 수 있습니다. 아래는 ContinuousCartPole 환경을 활용하며, 학습중 발생하는 로그 중 Mean Return에 해당하는 값이 에피소드가 증가함에 따라 함께 증가한다면, 학습이 잘 진행되는 것으로 판단할 수 있습니다. 이 환경에서 얻을 수 있는 최대 Mean Return은 1000.0입니다.

In [ ]:
from miscs.m0804.train import train

env_name = 'ContinuousCartPole-v0'
obs_dim = 5
act_dim = 1
policy = Policy(obs_dim, act_dim, clipping=0.2)
train(env_name, policy)